In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist


In [37]:
# Load MNIST data
(X_train, _), (_, _) = mnist.load_data()

# Normalize the images to range [-1, 1] for better GAN performance
X_train = X_train.astype(np.float32)
X_train = (X_train - 127.5) / 127.5  # Scale to [-1, 1]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)  # Reshape to (28, 28, 1) for CNN

# Batch size and random noise dimension
batch_size = 128
random_dim = 100


In [38]:
def build_generator():
    model = Sequential()

    # Fully connected layer, followed by reshaping into a 7x7x128 tensor
    model.add(layers.Dense(256, input_dim=random_dim, kernel_initializer='he_normal'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization(momentum=0.8))

    model.add(layers.Dense(28 * 28 * 1, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))

    return model

generator = build_generator()
generator.summary()


C:\Users\PC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 784)            │       803,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,493,520 (5.70 MB)

 Trainable params: 1,489,936 (5.68 MB)

 Non-trainable params: 3,584 (14.00 KB)

In [39]:
def build_discriminator():
    model = Sequential()

    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability for real/fake

    return model

discriminator = build_discriminator()
discriminator.summary()


C:\Users\PC\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │       803,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,460,225 (5.57 MB)

 Trainable params: 1,460,225 (5.57 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Compile discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Freeze discriminator when training the generator
discriminator.trainable = False

# Build the GAN model (stack generator and discriminator)
z = layers.Input(shape=(random_dim,))
img = generator(z)
valid = discriminator(img)

gan = tf.keras.models.Model(z, valid)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))


In [52]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Ensure optimizer and loss function are properly set
generator.optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator.optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

def train_gan(epochs, batch_size=128, save_interval=50):
    half_batch = batch_size // 2
    
    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_images = X_train[idx]
        real_labels = np.ones((half_batch, 1))  # Labels for real images

        noise = np.random.normal(0, 1, (half_batch, random_dim))  # Generate random noise
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))  # Labels for fake images

        with tf.GradientTape() as tape:
            real_preds = discriminator(real_images, training=True)
            fake_preds = discriminator(fake_images, training=True)
            real_loss = tf.keras.losses.binary_crossentropy(real_labels, real_preds)
            fake_loss = tf.keras.losses.binary_crossentropy(fake_labels, fake_preds)
            d_loss = 0.5 * (tf.reduce_mean(real_loss) + tf.reduce_mean(fake_loss))

        grads = tape.gradient(d_loss, discriminator.trainable_variables)
        if grads:
            discriminator.optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))
        else:
            print("No gradients found for discriminator!")

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, random_dim))  # Generate random noise
        valid_labels = np.ones((batch_size, 1))  # Labels for real images (generator tries to fool the discriminator)

        with tf.GradientTape() as tape:
            fake_images = generator(noise, training=True)
            fake_preds = discriminator(fake_images, training=True)
            g_loss = tf.keras.losses.binary_crossentropy(valid_labels, fake_preds)

        grads = tape.gradient(g_loss, generator.trainable_variables)
        if grads:
            generator.optimizer.apply_gradients(zip(grads, generator.trainable_variables))
        else:
            print("No gradients found for generator!")

        # If at save interval, save generated image samples
        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss}] [G loss: {g_loss}]")
            save_generated_images(epoch)


In [ ]:
# Train the GAN for 100 epochs and save images every 10 epochs
train_gan(epochs=100, batch_size=128, save_interval=10)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
No gradients found for discriminator!
0 [D loss: 0.6746596693992615] [G loss: [0.26229697 0.7054137  0.7698171  0.3495694  1.1023921  1.3419147
 0.786191   0.5291417  0.3174114  0.4155292  0.34580228 1.2444868
 0.69339913 0.35692355 0.3456124  0.8156133  1.0404136  0.6704359
 0.8168278  0.41530752 0.48353413 0.52472293 0.7405164  0.4916998
 0.23437145 1.2096282  0.41843084 0.40663218 0.3748266  0.3767918
 1.1369363  0.88504446 0.18679571 0.6067716  0.49696916 0.43707865
 0.910909   0.67259234 0.9882922  1.0675353  0.65867853 0.3178441
 0.4748052  0.32812914 0.75182724 0.37326118 0.7783415  0.33820483
 0.5481842  0.3959892  0.7188201  0.98854613 0.8988881  0.27509642
 0.15905543 0.47181788 0.3052786  0.8286857  0.84705037 0.70873356
 0.68169504 0.72951066 0.48046064 0.87700266 0.46829098 0.6303099
 0.69878966 0.6575031  0.916819   0.40890676 0.39407915 0.6034124
 0.37243968 0.43858787 0.3232011  0.91619855 1.3944914  0.3543238
 0.9171173  0.46617773